In [1]:
import sys

sys.argv = [""]
del sys

import argparse

In [2]:
def create_cfg():
    parser = argparse.ArgumentParser()
    for key, val in args.items():
        parser.add_argument(key, default=val)
        # print("--" + key, val)
    cfg = parser.parse_args()
    return cfg

In [10]:
args = {}

# Data args
args["--train_ds_dir"] = '../../../Datasets/Places365_Standard/data_large'
args["--val_ds_dir"] = '../../../Datasets/Places365_Standard/val_large'
args["--CKPT_DIR"] = '../../experiments/inpaint/ckpts/exp1'
args["--LOG_DIR"] = '../../experiments/inpaint/logs/exp1'
args["--SAMPLE_DIR"] = '../../experiments/inpaint/samples/exp1'

# Image and Mask args
args["--crop_size"] = (256, 256)
args["--mask_type"] = 'free_form'
args["--mask_num"] = 20
args["--max_angle"] = 10 
args["--max_len"] = 40
args["--max_width"] = 50 
args["--margin"] = (10, 10) 
args["--bbox_shape"] = (100, 100) 

# Training args
args["--epochs"] = 1
args["--batch_size"] =  10
args["--lr_g"] =  1e-4 
args["--lr_d"] = 1e-4 
args["--b1"] = 0.5
args["--b1"] = 0.999
args["--lambda_l1"] = 10
args["--lambda_perceptual"] = 10
args["--lambda_gan"] = 1 
args["--lr_decrease_epoch"] = 10
args["--lr_decrease_factor"] = 0.5 
args["--num_workers"] = 1
args["--pin_memory"] = True
args["--use_perceptualnet"] = True
args["--LOG_INTERVAL"] = 100
args["--SAVE_SAMPLES_INTERVAL"] = 5
args["--SAVE_SAMPLE_COUNT"] = 20


# Model args
args["--in_channels"] = 4
args["--out_channels"] = 3
args["--latent_channels"] = 64
args["--pad_type"] = "zero"
args["--activation"] = "elu"
args["--norm_d"] = "none"
args["--norm_g"] = "none"
args["--init_type"] = "kaiming"
args["--init_gain"] = 0.02
args["--use_cuda"] = True

In [11]:
cfg = create_cfg()

In [17]:
from inpaint.data import PlacesDataset
from torch.utils.data import DataLoader
from inpaint.core.discriminator import PatchDiscriminator
from inpaint.core.generator import GatedGenerator

In [13]:
train_ds = PlacesDataset(path_dir = cfg.train_ds_dir, batch_size=cfg.batch_size)

In [14]:
val_ds = PlacesDataset(path_dir = cfg.val_ds_dir, batch_size=cfg.batch_size)